This notebook is just a scratch pad for working out a pipeline to turn user text into a prediction. Once I have the outline here I'm going to refactor it into a script or module for the webapp. 

In [1]:
import os
import numpy as np
import pickle
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
model = load_model('model-development/models/cat-cross-model-e2.h5')
max_len = 200

In [3]:
test_string_civil = '''What's this you've said to me, my good friend? 
I'll have you know I graduated top of my class in conflict resolution, 
and I've been involved in numerous friendly discussions, 
and I have over 300 confirmed friends. 
I am trained in polite discussions and I'm the top mediator in the entire neighbourhood. 
You are worth more to me than just another target. 
I hope we will come to have a friendship never before seen on this Earth. 
Don't you think you might be hurting someone's feelings saying that over the internet? 
Think about it, my friend. As we speak I am contacting my good friends across the USA and your P.O. box is being traced right now so you better prepare for the greeting cards, friend. 
The greeting cards that help you with your hate. 
You should look forward to it, friend. I can be anywhere, anytime for you, and I can calm you in over seven hundred ways, 
and that's just with my chess set. Not only am I extensively trained in conflict resolution, 
but I have access to the entire group of my friends and I will use them to their full extent to start our new friendship. 
If only you could have known what kindness and love your little comment was about to bring you, 
maybe you would have reached out sooner. But you couldn't, you didn't, and now we get to start a new friendship, 
you unique person. I will give you gifts and you might have a hard time keeping up. You're finally living, friend.'''

test_string_toxic = '''What the fuck did you just fucking say about me, you little bitch? 
I’ll have you know I graduated top of my class in the Navy Seals, 
and I’ve been involved in numerous secret raids on Al-Quaeda, 
and I have over 300 confirmed kills. 
I am trained in gorilla warfare and I’m the top sniper in the entire US armed forces. 
You are nothing to me but just another target. 
I will wipe you the fuck out with precision the likes of which has never been seen before on this Earth, mark my fucking words. 
You think you can get away with saying that shit to me over the Internet? 
Think again, fucker. As we speak I am contacting my secret network of spies across the USA and your IP is being traced right now so you better prepare for the storm, maggot. 
The storm that wipes out the pathetic little thing you call your life. You’re fucking dead, kid. I can be anywhere, anytime, and I can kill you in over seven hundred ways, 
and that’s just with my bare hands. Not only am I extensively trained in unarmed combat, 
but I have access to the entire arsenal of the United States Marine Corps 
and I will use it to its full extent to wipe your miserable ass off the face of the continent, 
you little shit. If only you could have known what unholy retribution your little “clever” comment 
was about to bring down upon you, maybe you would have held your fucking tongue. 
But you couldn’t, you didn’t, and now you’re paying the price, you goddamn idiot. 
I will shit fury all over you and you will drown in it. You’re fucking dead, kiddo.'''

In [4]:
tokenizer = pickle.load(open('model-development/models/comment-tokenizer.pkl','rb'))

In [5]:
test_seq = tokenizer.texts_to_sequences([test_string_civil])

In [6]:
test_arr = pad_sequences(np.array(test_seq), maxlen=max_len)

In [7]:
prediction = model.predict(test_arr)[0]

In [8]:
prediction

array([ 0.98149121,  0.01850882], dtype=float32)

In [9]:
toxic_prob = prediction[1]
civil_prob = prediction[0]

In [10]:
if toxic_prob > civil_prob:
    print(f'this comment is more likely to be toxic({toxic_prob}) than civil({civil_prob})')
else:
    print(f'this comment is more likely to be civil({civil_prob}) than toxic({toxic_prob})')
assert toxic_prob + civil_prob == 1

this comment is more likely to be civil(0.981491208076477) than toxic(0.018508821725845337)


In [11]:
def Score_Comment(comment, model, tokenizer, expected_len):
    tokens = tokenizer.texts_to_sequences([comment])
    arr = pad_sequences(tokens, maxlen=expected_len)
    pred = model.predict(arr)[0]
    return pred[1]

In [12]:
Score_Comment(test_string_toxic, model=model, tokenizer=tokenizer, expected_len=200)

0.99979335

In [14]:
def Report_Comment(comment, model, tokenizer, expected_len):
    score = Score_Comment(comment, model, tokenizer, expected_len)
    return f"the string:\n'''{comment}'''\nHas a {score*100}% probability of being toxic"

In [15]:
print(Report_Comment(test_string_civil, model=model, tokenizer=tokenizer, expected_len=200))

the string:
'''What's this you've said to me, my good friend? 
I'll have you know I graduated top of my class in conflict resolution, 
and I've been involved in numerous friendly discussions, 
and I have over 300 confirmed friends. 
I am trained in polite discussions and I'm the top mediator in the entire neighbourhood. 
You are worth more to me than just another target. 
I hope we will come to have a friendship never before seen on this Earth. 
Don't you think you might be hurting someone's feelings saying that over the internet? 
Think about it, my friend. As we speak I am contacting my good friends across the USA and your P.O. box is being traced right now so you better prepare for the greeting cards, friend. 
The greeting cards that help you with your hate. 
You should look forward to it, friend. I can be anywhere, anytime for you, and I can calm you in over seven hundred ways, 
and that's just with my chess set. Not only am I extensively trained in conflict resolution, 
but I have